In [9]:
import telebot
import feedparser
import re
from bs4 import BeautifulSoup


token = '980878523:AAEUYxMijMvEJTuS55crcbQc10JLbF_ApM4'
bot = telebot.TeleBot(token)


#переменная, в которую записывается последнее действие пользователя
last_click = None

# стартовая клавиатура
start_kbrd = telebot.types.ReplyKeyboardMarkup(True, True)
start_kbrd.row('Выбрать категорию', 'Выбрать ключевые слова')
start_kbrd.row('Добавить источник')
start_kbrd.row('Вывести подборку новостей')
start_kbrd.row('Выход')

# клавиатура с выбором категорий
categories_markup_kbrd = telebot.types.ReplyKeyboardMarkup(True, True)
categories_markup_kbrd.row('Политика', 'Спорт')
categories_markup_kbrd.row('Наука', 'Искусство')
categories_markup_kbrd.row('Отмена')

# клавиатура общего назначения (отмены)
cancel_markup_kbrd = telebot.types.ReplyKeyboardMarkup(True, True)
cancel_markup_kbrd.row('Отмена')


# список источников
sources = {
    'Политика':[
        'https://www.vedomosti.ru/rss/rubric/politics',
        'https://lenta.ru/rss/news'
    ],
    'Спорт':[
        'http://www.sports.ru/rss/all_news.xml'
    ],
    'Наука':[
        'https://habr.com/ru/rss/news/'
    ],
    'Культура':[
        
    ]
}

# список пользователей и их предпочтений
user_list = {
    "users": {}, 
    "ver": 1
}

# хранилище новостей
news_storage = {
    'Политика':[],
    'Спорт':[],
    'Наука':[],
    'Культура':[]
}

# запуск бота
@bot.message_handler(commands=['start', 'help'])
def start_message(message):
    global start_kbrd

    bot.send_message(message.chat.id,
                     'Привет!  :)\n\n' +
                     'Я помогу тебе получать только самые интересные и важные новости' +
                     '\n\nВот, что я пока что умею:\n' +
                     '✔ подбирать новости из установленных источников\n' +
                     '✔ добавлять твои собственные источники' +
                     '✔ выбирать интересующие тебя ключевые слова', 
                     reply_markup=start_kbrd)

    #record_data('id', message.from_user.id)

# основной обработчик событиий
@bot.message_handler(content_types=['text'])
def send_message(message):
    global last_click, start_kbrd, categories_markup_kbrd, cancel_markup_kbrdб, sources

    if message.text == 'Выход':
        bot.send_message(message.chat.id, 'На сегодня, похоже, хватит новостей')

    elif message.text == 'Выбрать категорию':
        last_click = 'choose category'
        bot.send_message(message.chat.id, 'Выбери что-нибудь из этого:', 
                         reply_markup=categories_markup_kbrd)
        
    elif message.text == 'Выбрать ключевые слова':
        last_click = 'choose keywords'
        bot.send_message(message.chat.id, 'Введите ключевые слова через запятую:', 
                         reply_markup=cancel_markup_kbrd)
        
    elif message.text == 'Добавить источник'
        last_click = 'add source'
        bot.send_message(message.chat.id, 'Введите ссылку на rss/atom ленту и категорию в формате: \n' +
                        'http://www.[название сайта].ru/[название ленты].xml, [название категории]')
    
    elif message.text == 'Вывести подборку новостей':
        last_click = 'None'
    
    # обработка 'шагов назад'
    elif message.text == 'Отмена':
        if last_click == 'choose category':
            bot.send_message(message.chat.id, 'Вы отменили выбор категории. Выберите дальнейшее действие.',
                             reply_markup=start_kbrd)
        
        elif last_click == 'choose keywords':
            bot.send_message(message.chat.id, 'Вы отменили выбор ключевых слов. Выберите дальнейшее действие.',
                             reply_markup=start_kbrd)
            
        elif last_click == 'add source':
            bot.send_message(message.chat.id, 'Вы отменили добавление нового источника. Выберите дальнейшее действие.',
                             reply_markup=start_kbrd)
    
    # обработка пользовательского ввода        
    elif message.text in sources.keys():
        
        #record_data('categories', message.text, message.chat.id)

    #else:
    #    bot.send_message(message.chat.id, message.text)
    #    print(message.text)



def add_user(id):
    global user_list
    user_list['users'].append(id: {categories: [], keywords: []})

def add_category(id, category):
    global user_list
    if category not in user_list['users'][id][categories]:
        user_list['users'][id][categories].append(category)
    else:
        return None
    
def add_keyword(id, keyword):
    global user_list
    if category not in user_list['users'][id][keywords]:
        user_list['users'][id][keywords].append(keyword)
    else:
        return None

def display_news(id):
    global user_list, news_storage
    categories = user_list['users'][id][categories]
    keywords = user_list['users'][id][keywords]
    
    

# парс определнной ссылки (ленты)    
def get_info(link, category):
    global news_storage
    
    news_line = feedparser.parse(link)
    for news in news_line.entries:
        content = cleanhtml(news.summary)
        if 'document.write' in content:
            content = content[: content.index('document.write') - 1]
        clean_ver = {'title': news.title, 'link': news.links[0]['href'], 'content': content}
        news_storage[category].append(clean_ver)   

# очистка текста от мусора 
def cleanhtml(raw_html):
    
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, '', raw_html)
    return ' '.join(cleantext.split())

# апдейт хранилиа новостей
def update_news(src_list):
    
    for category in src_list:
        for link in sources[category]:
            get_info(link, category)
    
# запись в json файл
    
#def record_data(cat, data, ident=None):
#    import json
#    with open('data_file.json', 'r') as read_obj:
#        current = json.load(read_obj)
#        if cat == 'id':
#            current['users'][data] = {"categories": [], "keywords": []}
#        else:
#            current['users'][str(ident)][cat].append(data)
#    with open('data_file.json', 'w') as write_obj:
#        json.dump(current, write_obj, ensure_ascii=False)


#bot.polling()